In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import sys
import QuantLib as ql

In [2]:
#  SW20251113_SOFR O/4.2587  matu : 2025-11-13
#19/12/2024
# floating, ANNU
# fixe ANNU : 2025
# 1 PAIEMENT
#24/1/2025  ---> 
#24/1/2026
# quotation 100.016363
# "startDate" : "2024-11-21",
# swap rate : 4.2587

In [3]:
today = ql.Date(19, ql.December, 2024)
ql.Settings.instance().evaluationDate = today

bps = 1e-4

In [4]:
[today, today + ql.Period(50, ql.Years)]

[Date(19,12,2024), Date(19,12,2074)]

In [5]:
def nparray2listQlDates(today,dates):
    qldates = []
    for i in range(len(dates)):
        if dates[i]>=0:
            qldates.append(today+ql.Period(dates[i], ql.Years))
        else:
            print(- dates[i])
            qldates.append(today-ql.Period(-dates[i], ql.Years))
    return qldates

In [6]:
ibordates= [0,1,2,3,4,5,6,7,8,9,10,15,20,30]
ibordatesql2 = nparray2listQlDates(today,ibordates)
iborvaleursnp= np.array([4.57,4.418842,4.588199,4.383796,4.442341,4.472094,4.487725,
                       4.603022,4.49304931,4.55046352,4.56296427,4.817077,4.969355, 4.76199173])/100
iborvaleurs=iborvaleursnp.tolist()
sofr_curve = ql.ZeroCurve(
    ibordatesql2,
    iborvaleurs,
    ql.Actual365Fixed()
)
sofr_curve

<QuantLib.QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x7f51a9948030> >

In [7]:
ibordatesql2 

[Date(19,12,2024),
 Date(19,12,2025),
 Date(19,12,2026),
 Date(19,12,2027),
 Date(19,12,2028),
 Date(19,12,2029),
 Date(19,12,2030),
 Date(19,12,2031),
 Date(19,12,2032),
 Date(19,12,2033),
 Date(19,12,2034),
 Date(19,12,2039),
 Date(19,12,2044),
 Date(19,12,2054)]

In [8]:
iborvaleurs

[0.045700000000000005,
 0.04418842,
 0.045881990000000004,
 0.04383796,
 0.044423409999999997,
 0.04472094,
 0.04487725,
 0.046030220000000004,
 0.044930493099999996,
 0.045504635200000004,
 0.0456296427,
 0.04817077,
 0.04969355,
 0.0476199173]

In [9]:
[today, today + ql.Period(50, ql.Years)]

[Date(19,12,2024), Date(19,12,2074)]

In [10]:
sofr_handle = ql.YieldTermStructureHandle(sofr_curve)
sofr = ql.Sofr(sofr_handle)

In [11]:
d = ql.Date(21, ql.November, 2024)
while d <= today:
    if sofr.isValidFixingDate(d):
        sofr.addFixing(d, 4.57/100)
    d += 1

In [12]:
sofr.fixing(ql.Date(19, ql.December, 2025))

0.045275737731582666

In [13]:
start_date = ql.Date(21, ql.November, 2024)
end_date = start_date + ql.Period(1, ql.Years) 
coupon_tenor = ql.Period(1, ql.Years)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
convention = ql.Following
rule = ql.DateGeneration.Forward
end_of_month = False

fixed_rate = 4.2587 /100
fixed_day_counter = sofr.dayCounter()

schedule = ql.Schedule(
    start_date,
    end_date,
    coupon_tenor,
    calendar,
    convention,
    convention,
    rule,
    end_of_month,
)
swap = ql.OvernightIndexedSwap(
    ql.Swap.Payer,
    100,
    schedule,
    fixed_rate,
    fixed_day_counter,
    sofr,
)

In [14]:
swap.setPricingEngine(
    ql.DiscountingSwapEngine(sofr_handle)
)

In [15]:
fixed_rate

0.042587

In [16]:
swap.NPV()

0.21945425408937247

In [17]:
swap.fixedLegNPV()

-4.1447877732828164

In [18]:
swap.overnightLegNPV()

4.364242027372189

In [19]:
ratio=100*swap.overnightLegNPV()/swap.fixedLegNPV()
ratio

-105.29470424285577

In [20]:
data = []
for cf in swap.fixedLeg():
    coupon = ql.as_fixed_rate_coupon(cf)
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )

In [21]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"November 21st, 2025",4.26%,1.013889,4.32


In [30]:
data = []
for cf in swap.overnightLeg():
    coupon = ql.as_floating_rate_coupon(cf)
    print(coupon.date())
    print(coupon.accrualPeriod())
    print(coupon.amount())    
    data.append(
        (
            coupon.date(),
            coupon.rate(),
            coupon.accrualPeriod(),
            coupon.amount(),
        )
    )
data

November 21st, 2025
1.0138888888888888
4.546465924723697


[(Date(21,11,2025),
  0.04484185569590496,
  1.0138888888888888,
  4.546465924723697)]

In [23]:
pd.DataFrame(
    data, columns=["date", "rate", "tenor", "amount"]
).style.format(
    {"amount": "{:.2f}", "rate": "{:.2%}"}
)

,date,rate,tenor,amount
0,"November 21st, 2025",4.48%,1.013889,4.55


In [24]:
swap.fairRate()

0.044841855695904986

In [31]:
data=(100.016363,103.7603494, abs(ratio))
pd.DataFrame(data)


,0
0,100.016363
1,103.760349
2,105.294704


In [27]:
data


[100.016363, 103.7603494, -105.29470424285577]